<img align="right" src="../img/auvisus.svg" width="100" >


# DEtection TRansformer Network

---

## 1. Import Modules

In [43]:
%load_ext autoreload
%autoreload 2

from detr_models.detr.model import DETR
from detr_models.detr.train import get_image_information

from detr_models.detr.data_feeder import DataFeeder
from detr_models.detr.uuid_iterator import UUIDIterator
from detr_models.detr.config import DefaultDETRConfig
from detr_models.backbone.backbone import Backbone

from detr_models.detr.data_feeder import create_positional_encodings
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

## 2. Load Model

In [44]:
path = "/Users/Schlendrikovic/Documents/Arbeit/Auvisus/Code/detr-tensorflow/pretrained_models/example_model"
model = tf.keras.models.load_model(path)
model.summary()

Model: "DETR"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Batch_Input (InputLayer)        [(None, 270, 480, 3) 0                                            
__________________________________________________________________________________________________
Backbone (Model)                (None, 17, 30, 1024) 8589184     Batch_Input[0][0]                
__________________________________________________________________________________________________
tf_op_layer_StopGradient (Tenso (None, 17, 30, 1024) 0           Backbone[1][0]                   
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa (None, 17, 30, 1024) 0           Backbone[1][0]                   
_______________________________________________________________________________________________

layer_normalization_24 (LayerNo (None, 100, 256)     512         tf_op_layer_Decoder_Add_2_4[0][0]
__________________________________________________________________________________________________
tf_op_layer_Decoder_Add_4_4 (Te (None, 100, 256)     0           layer_normalization_24[0][0]     
                                                                 embedding[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_Shape_31 (TensorFlo (3,)                 0           tf_op_layer_Decoder_Add_4_4[0][0]
__________________________________________________________________________________________________
tf_op_layer_strided_slice_31 (T ()                   0           tf_op_layer_Shape_31[0][0]       
__________________________________________________________________________________________________
tf_op_layer_Decoder_Add_3_4 (Te (None, 510, 256)     0           layer_normalization_11[0][0]     
          

---

## 3. Load Data

In [62]:
# Load Config
config = DefaultDETRConfig()

fm_shape=model.get_layer("Backbone").output.shape[1::]
positional_encodings = create_positional_encodings(fm_shape, config.dim_transformer//2, batch_size=1)

image_path = "/Users/Schlendrikovic/Documents/Arbeit/Auvisus/Data/Detector/test/images/0a2cc362d894e89134db26ea1aa852a1b7397404_00_00186.jpg"
image = img_to_array(load_img(image_path), dtype=np.float32)

image = np.expand_dims(image,0)

print(positional_encodings.shape)
print(image.shape)

(1, 510, 256)
(1, 270, 480, 3)


In [63]:
for input in model.inputs:
    print("Expected Input: {}".format(input))

Expected Input: Tensor("Batch_Input_1:0", shape=(None, 270, 480, 3), dtype=float32)
Expected Input: Tensor("Positional_Encodings_Input_1:0", shape=(None, 510, 256), dtype=float32)


---

## 4. Inference

In [64]:
%%time
detr_scores, detr_bbox = model([image, positional_encodings], training=False)

print(detr_scores.shape)
print(detr_bbox.shape)

(1, 100, 5)
(1, 100, 4)
CPU times: user 1.99 s, sys: 539 ms, total: 2.53 s
Wall time: 1.11 s


---

*This Notebook was created by: [auvisus GmbH](https://www.auvisus.com/)*